<a href="https://colab.research.google.com/github/Nathan-Roll1/PSST/blob/main/Tutorials/PSST_Q_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Necessary Packages

In [ ]:
!python -m pip install optimum[onnxruntime]@git+https://github.com/Nathan-Roll1/optimum-CPU-PSST
# Custom fork to overcome CuPy requirement on CPU

!pip install -U transformers torch

Import modules

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoFeatureExtractor
from optimum.onnxruntime import (
    AutoQuantizationConfig,
    ORTModelForSpeechSeq2Seq,
    ORTQuantizer
)

Create inference Pipeline

In [ ]:
model_dir = "NathanRoll/psst-medium-en-q"

model = ORTModelForSpeechSeq2Seq.from_pretrained(model_dir, provider="CPUExecutionProvider")
tokenizer = AutoTokenizer.from_pretrained(model_dir)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_dir)
cls_pipeline_onnx = pipeline("automatic-speech-recognition", model=model, tokenizer=tokenizer, feature_extractor=feature_extractor)

Generate transcription! (*!!!!! = IU Boundary*)

In [5]:
inference_file = "gettysburg.wav"
cls_pipeline_onnx(inference_file) #22s #41s

{'text': 'Four score and seven years ago!!!!! our fathers brought forth on this continent!!!!! a new nation!!!!! conceived in liberty!!!!! and dedicated to the proposition!!!!! that all men are created equal!!!!! Now we are engaged in a great civil war!!!!! testing whether that nation!!!!! or any nation so conceived and so dedicated!!!!! can long endure'}

Also works with librosa, useful to feed audio longer than 30s!

In [12]:
import librosa
y, sr = librosa.load('gettysburg.wav')

In [14]:
cls_pipeline_onnx(y)

{'text': 'Four score and seven years ago!!!!! our fathers brought forth on this continent!!!!! a new nation!!!!! conceived in liberty!!!!! and dedicated to the proposition!!!!! that all men are created equal!!!!! Now we are engaged in a great civil war!!!!! testing whether that nation!!!!! or any nation so conceived and so dedicated!!!!! can long endure'}